---
layout: post  
current: post  
cover:  assets/built/images/항공사만족도분류.png  
navigation: True  
title: 항공사 만족도 분류 Modeling  
date: 2023-03-06 00:00:00 +0900  
tags: [project]  
class: post-template  
subclass: 'post tag-python'  
author: chanp5660  
---


# 항공사 만족도 분류 분류 모델 구현

1. train set과 test set을 이용하여 모델링을 하고 평가한다.
    - 로지스틱 회귀분석, 결정트리, 랜덤 포레스트, 서포트 벡터 머신, XGBoosting
2. train set에 Unlabeled set을 추가하여 test set으로 평가한다.
    - 가장 성능이 좋았던 XGBoosting으로 실행
    - 하이퍼 파라미터를 조정하여 성능 향상

## 모델 선택

- 목표 : 승객의 설문 조사 데이터로부터 만족 여부(satisfaction)를 판단
- satisfaction은 [Satisfaction : 1, neutral or dissatisfaction : 0]으로 이진분류 알고리즘을 사용


| Algorithm | Accuracy_val | cv=5 | Accuracy_test |
|:----------|----------:|----------:|----------:|
| 로지스틱 회귀분석 | 0.894 | 0.89 | 0.892 |
| 결정트리 | 0.897 | 0.888 | 0.892 |
| 랜덤 포레스트 | 0.931 | 0.931 | 0.934 |
| 서포트 벡터 머신 | 0.935 | 0.934 | 0.937 |
| XGBoosting | 0.934 | 0.979 | 0.956 |
| Unlabeled 데이터 추가+ XGBoosting|||0.987|


> 테스트 데이터에 가장 성능이 좋은 모델은 정확도가 **0.956**인 **XGBoosting** 모델이다.
Unlabeled data가 추가된 데이터의 학습 모델의 정확도는 **0.987**로 **XGBoosting** 모델이다.

- 라이브러리 설정 및 전처리

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# 데이터 로드
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# 전처리 (결측치 제거)
train.dropna(axis=1,inplace = True)
train.drop(columns=['id'],inplace = True)
test = test[train.columns]
    
# 전처리 (스케일링)
scaler = MinMaxScaler()
train[["Age","Flight Distance"]] = pd.DataFrame(scaler.fit_transform(train[["Age","Flight Distance"]]), columns=["Age","Flight Distance"])
test[["Age","Flight Distance"]] = pd.DataFrame(scaler.transform(test[["Age","Flight Distance"]]), columns=["Age","Flight Distance"])

# 범주형 변수를 원핫인코딩으로 변환하기
ohe = OneHotEncoder()
categorical_features = ['Gender', 'Customer Type', 'Type of Travel', 'Class', "Inflight wifi service","Departure/Arrival time convenient","Ease of Online booking","Gate location","Food and drink"]
ohe.fit(train[categorical_features]) # 특정 열(범주형)만 원핫인코딩
encoded_categorical = ohe.transform(train[categorical_features]).toarray() 
encoded_categorical_test = ohe.transform(test[categorical_features]).toarray() 

# 원핫인코딩 된 데이터 데이터 프레임으로 합치기
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=ohe.get_feature_names(categorical_features))
encoded_categorical_df_test = pd.DataFrame(encoded_categorical_test, columns=ohe.get_feature_names(categorical_features))

# 전처리; 숫자형 변수와 원핫인코딩된 범주형 변수를 합치기
X = pd.concat([train.drop(categorical_features + ['satisfaction'], axis=1), encoded_categorical_df], axis=1)
y = train['satisfaction']

X_test = pd.concat([test.drop(categorical_features + ['satisfaction'], axis=1), encoded_categorical_df_test], axis=1)
y_test = test['satisfaction']

# 학습, 테스트 데이터 분리 7:3
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size = 0.3, random_state = 123)

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### 로지스틱 회귀분석

#### test, val의 정확도

In [2]:
from sklearn.linear_model import LogisticRegression
# 로지스틱 회귀 모델 인스턴스 생성 및 학습
lr = LogisticRegression(max_iter = 1000,random_state = 123) # 경사하강 반복횟수 필요에 따라 늘림
lr.fit(X_train,y_train)

# 예측
y_val_pred = lr.predict(X_val)

# 성능 평가
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Acc:{accuracy:.3}")
conf_matrix = confusion_matrix(y_val, y_val_pred)
print("Confusion matrix:\n", conf_matrix)

Acc:0.894
Confusion matrix:
 [[1475  221]
 [  97 1207]]


#### 교차검증

In [3]:
# 로지스틱 회귀 모델 인스턴스 생성 및 학습
lr = LogisticRegression(max_iter = 1000,random_state = 123) # 경사하강 반복횟수 필요에 따라 늘림
scores = cross_val_score(lr, X_train, y_train, cv=5)

# 교차 검증 결과 출력
print(f"교차 검증 점수: {scores}")
print(f"교차 검증 평균 점수: {scores.mean():.3}")

교차 검증 점수: [0.87642857 0.89714286 0.89714286 0.89357143 0.88357143]
교차 검증 평균 점수: 0.89


#### Test.csv

In [4]:
# 로지스틱 회귀 모델 인스턴스 생성 및 학습
lr = LogisticRegression(max_iter = 1000,random_state = 123) 
lr.fit(X,y)

# 예측
y_pred = lr.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Acc:{accuracy:.3}")

Acc:0.892


### 결정트리

#### test, val의 정확도

In [5]:
from sklearn.tree import DecisionTreeClassifier

# 이진트리 분류 모델 인스턴스 생성 및 학습
clf = DecisionTreeClassifier(random_state = 123)
clf.fit(X_train, y_train)

# 예측
y_val_pred = clf.predict(X_val)

# 성능 평가
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Acc:{accuracy:.3}")
conf_matrix = confusion_matrix(y_val, y_val_pred)
print("Confusion matrix:\n", conf_matrix)

Acc:0.897
Confusion matrix:
 [[1533  163]
 [ 146 1158]]


#### 교차검증

In [6]:
# 결정트리 회귀 모델 인스턴스 생성 및 학습
scores = cross_val_score(clf, X_train, y_train, cv=5)

# 교차 검증 결과 출력
print(f"교차 검증 점수: {scores}")
print(f"교차 검증 평균 점수: {scores.mean():.3}")

교차 검증 점수: [0.88714286 0.88928571 0.885      0.88857143 0.89071429]
교차 검증 평균 점수: 0.888


#### Test.csv

In [7]:
# 이진트리 분류 모델 인스턴스 생성 및 학습
clf = DecisionTreeClassifier(random_state = 123)
clf.fit(X, y)

# 예측
y_pred = clf.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Acc:{accuracy:.3}")

Acc:0.892


## 랜덤 포레스트

#### test, val의 정확도

In [8]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 모델 학습하기
clf = RandomForestClassifier(n_estimators=100, random_state=123)
clf.fit(X_train, y_train)

# 예측
y_val_pred = clf.predict(X_val)

# 성능 평가
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Acc:{accuracy:.3}")
conf_matrix = confusion_matrix(y_val, y_val_pred)
print("Confusion matrix:\n", conf_matrix)

Acc:0.931
Confusion matrix:
 [[1589  107]
 [  99 1205]]


#### 교차검증

In [9]:
# 랜덤포레스트 회귀 모델 인스턴스 생성 및 학습
scores = cross_val_score(clf, X_train, y_train, cv=5)

# 교차 검증 결과 출력
print(f"교차 검증 점수: {scores}")
print(f"교차 검증 평균 점수: {scores.mean():.3}")

교차 검증 점수: [0.92571429 0.93       0.93428571 0.93642857 0.92785714]
교차 검증 평균 점수: 0.931


#### Test.csv

In [10]:
# 랜덤포레스트 모델 학습하기
clf = RandomForestClassifier(n_estimators=100, random_state=123)
clf.fit(X, y)

# 예측
y_pred = clf.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Acc:{accuracy:.3}")

Acc:0.934


## 서포트 벡터 머신

#### test, val의 정확도

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# 랜덤포레스트 모델 학습하기
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
clf = GridSearchCV(SVC(), param_grid)
clf.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력하기
print('Best parameters:', clf.best_params_)

# 예측
y_val_pred = clf.predict(X_val)

# 성능 평가
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Acc:{accuracy:.3}")
conf_matrix = confusion_matrix(y_val, y_val_pred)
print("Confusion matrix:\n", conf_matrix)

Best parameters: {'C': 1, 'kernel': 'rbf'}
Acc:0.935
Confusion matrix:
 [[1573  123]
 [  72 1232]]


#### 교차검증

In [12]:
# 랜덤포레스트 회귀 모델 인스턴스 생성 및 학습
scores = cross_val_score(clf, X_train, y_train, cv=5)

# 교차 검증 결과 출력
print(f"교차 검증 점수: {scores}")
print(f"교차 검증 평균 점수: {scores.mean():.3}")

교차 검증 점수: [0.93785714 0.93214286 0.93714286 0.93357143 0.92785714]
교차 검증 평균 점수: 0.934


#### Test.csv

In [13]:
# 랜덤포레스트 모델 학습하기
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
clf = GridSearchCV(SVC(), param_grid)
clf.fit(X, y)

# 최적의 하이퍼파라미터 출력하기
print('Best parameters:', clf.best_params_)

# 예측
y_pred = clf.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Acc:{accuracy:.3}")

Best parameters: {'C': 1, 'kernel': 'rbf'}
Acc:0.937


## XGBoosting

#### test, val의 정확도

In [29]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

#  하이퍼 파라미터 설정
param_grid = {'learning_rate': [0.01,0.1,0.5], 
              'max_depth': [1,3,5],
              "n_estimators" : [100, 200]}

# 모델 인스턴스 생성 및 학습
xgb_model = XGBClassifier(objective = 'binary:logistic')
grid_acc = GridSearchCV(estimator = xgb_model, param_grid = param_grid,
                        scoring="roc_auc", cv = 5, verbose = 0 )

grid_acc.fit(X_train, y_train)

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:59:17] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:59:18] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:59:18] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:59:18] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:59:18] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:59:50] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:59:51] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight

In [31]:
# 최적의 하이퍼파라미터 출력하기
print('Best parameters:', grid_acc.best_params_)

# 예측
y_val_pred = grid_acc.predict(X_val)

# 성능 평가
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Acc:{accuracy:.3}")
conf_matrix = confusion_matrix(y_val, y_val_pred)
print("Confusion matrix:\n", conf_matrix)

Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Acc:0.934
Confusion matrix:
 [[1579  117]
 [  80 1224]]


#### 교차검증

In [32]:
# XGBoosting 회귀 모델 인스턴스 생성 및 학습
scores = cross_val_score(grid_acc, X_train, y_train, cv=5)

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:00:26] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:00:26] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\x

[17:00:26] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:00:27] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:00:27] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:00:27] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\s

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[17:03:23] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:03:24] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:03:25] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
교차 검증 점수: [0.97792048 0.97848745 0.98238895 0.98132744 0.97357578]
교차 검증 평균 점수: 0.979


In [33]:
# 교차 검증 결과 출력
print(f"교차 검증 점수: {scores}")
print(f"교차 검증 평균 점수: {scores.mean():.3}")

교차 검증 점수: [0.97792048 0.97848745 0.98238895 0.98132744 0.97357578]
교차 검증 평균 점수: 0.979


#### Test.csv

In [34]:
#  하이퍼 파라미터 설정
param_grid = {'learning_rate': [0.01,0.1,0.5], 
              'max_depth': [1,3,5],
              "n_estimators" : [100, 200]}

# 모델 인스턴스 생성 및 학습
xgb_model = XGBClassifier(objective = 'binary:logistic')
grid_acc = GridSearchCV(estimator = xgb_model, param_grid = param_grid,
                        scoring="roc_auc", cv = 5, verbose = 0 )

grid_acc.fit(X, y)

[17:03:25] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:03:26] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:03:26] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:04:21] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:04:22] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight

In [35]:
# 최적의 하이퍼파라미터 출력하기
print('Best parameters:', grid_acc.best_params_)

# 예측
y_val_pred = grid_acc.predict(X_val)

# 성능 평가
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Acc:{accuracy:.3}")

Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Acc:0.956


# 3번 과제 Unlabeled의 Satisfaction 예측

In [274]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd

# 데이터 로드
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
Unlabeled = pd.read_csv("unlabeled.csv")
Unlabeled["satisfaction"] = ["neutral or dissatisfied"]+["satisfied"]*(len(Unlabeled)-1) # 예측하기 위해 임의로 satisfaction 생성

# 전처리 (결측치 제거)
train.dropna(axis=1,inplace = True)
train.drop(columns=['id'],inplace = True)
test = test[train.columns]
Unlabeled = Unlabeled[train.columns]

# 전처리 (스케일링)
scaler = MinMaxScaler()
train[["Age","Flight Distance"]] = pd.DataFrame(scaler.fit_transform(train[["Age","Flight Distance"]]), columns=["Age","Flight Distance"])
test[["Age","Flight Distance"]] = pd.DataFrame(scaler.transform(test[["Age","Flight Distance"]]), columns=["Age","Flight Distance"])
Unlabeled[["Age","Flight Distance"]] = pd.DataFrame(scaler.transform(Unlabeled[["Age","Flight Distance"]]), columns=["Age","Flight Distance"])

# 전처리 train 데이터에 Gate location 에 0 점이 없으므로 인코딩에 문제가 생긴다 데이터 중 하나를 바꾸어 준다.
train.loc[0,["Gate location"]] = 0

# 범주형 변수를 원핫인코딩으로 변환하기
ohe = OneHotEncoder()
categorical_features = ['Gender', 'Customer Type', 'Type of Travel', 'Class', "Inflight wifi service","Departure/Arrival time convenient","Ease of Online booking","Gate location","Food and drink"]
ohe.fit(train[categorical_features]) # 특정 열(범주형)만 원핫인코딩
encoded_categorical = ohe.transform(train[categorical_features]).toarray() 
encoded_categorical_test = ohe.transform(test[categorical_features]).toarray() 
encoded_categorical_Unlabeled = ohe.transform(Unlabeled[categorical_features]).toarray() 

# 원핫인코딩 된 데이터 데이터 프레임으로 합치기
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=ohe.get_feature_names(categorical_features))
encoded_categorical_df_test = pd.DataFrame(encoded_categorical_test, columns=ohe.get_feature_names(categorical_features))
encoded_categorical_df_Unlabeled = pd.DataFrame(encoded_categorical_Unlabeled, columns=ohe.get_feature_names(categorical_features))

# 전처리; 숫자형 변수와 원핫인코딩된 범주형 변수를 합치기
X = pd.concat([train.drop(categorical_features + ['satisfaction'], axis=1), encoded_categorical_df], axis=1)
y = train['satisfaction']

X_test = pd.concat([test.drop(categorical_features + ['satisfaction'], axis=1), encoded_categorical_df_test], axis=1)
y_test = test['satisfaction']

X_Unlabeled = pd.concat([Unlabeled.drop(categorical_features + ['satisfaction'], axis=1), encoded_categorical_df_Unlabeled], axis=1)
y_Unlabeled = Unlabeled['satisfaction']

In [287]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

#  하이퍼 파라미터 설정
param_grid = {'learning_rate': [0.01,0.1,0.5], 
              'max_depth': [1,3,5],
              "n_estimators" : [100, 200]}

# 모델 인스턴스 생성 및 학습
xgb_model = XGBClassifier(objective = 'binary:logistic')
grid_acc = GridSearchCV(estimator = xgb_model, param_grid = param_grid,
                        scoring="roc_auc", cv = 5, verbose = 0 )

grid_acc.fit(X, y)

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:48:16] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:48:16] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:49:17] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:49:18] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight

In [297]:
# 예측 
y_Unlabeled_pred = grid_acc.predict(X_Unlabeled) 

# 기존 데이터 satisfaction 열을 y_Unlabeled_pred 열로 수정
New_Unlabeled = X_Unlabeled.copy()

# 결합된 새로운 X,y의 데이터프레임을 생성
New_X = pd.concat((X,New_Unlabeled), axis = 0)
New_y = np.concatenate((y,y_Unlabeled_pred), axis=0)

In [301]:
# 학습, 테스트 데이터 분리 7:3
X_train,X_val,y_train,y_val = train_test_split(New_X, New_y, test_size = 0.3, random_state = 123)

In [302]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

#  하이퍼 파라미터 설정
param_grid = {'learning_rate': [0.001, 0.01,0.1,0.5], 
              'max_depth': [1,3,5],
              "n_estimators" : [100, 200, 300]}

# 모델 인스턴스 생성 및 학습
xgb_model = XGBClassifier(objective = 'binary:logistic')
grid_acc = GridSearchCV(estimator = xgb_model, param_grid = param_grid,
                        scoring="roc_auc", cv = 5, verbose = 0 )

grid_acc.fit(X_train, y_train)

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:57:57] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:57:58] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:16:21] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\master\anaconda3\envs\chanp5660\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:16:33] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight

In [303]:
# 최적의 하이퍼파라미터 출력하기
print('Best parameters:', grid_acc.best_params_)

# 예측
y_val_pred = grid_acc.predict(X_val)

# 성능 평가
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Acc:{accuracy:.3}")

Best parameters: {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 100}
Acc:0.987
